<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/PJH/UnixCoder_1_N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap

In [3]:
!pip install datasets

In [4]:
import pandas as pd

In [5]:
#df = pd.read_csv('/content/drive/MyDrive/Dataset/prototype5.csv')
df = pd.read_csv('/content/drive/MyDrive/MSR_1vs1_23000.csv')

In [6]:
df

,CWE ID,CVE ID,CODE,vul,lang,IDX,chmod(),assert(),atoi(),atof(),...,array_index_vulnerability,int,short,long,long long,unsigned int,unsigned short,unsigned long,unsigned long long,line_count
0,CWE-119,CVE-2015-8382,static PHP_FUNCTION(preg_match)\n{\n\tphp_do_p...,0,C,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,CWE-119,CVE-2015-8382,static PHP_FUNCTION(preg_match_all)\n{\n\tphp_...,0,C,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2,CWE-119,CVE-2015-8382,static PHP_FUNCTION(preg_replace)\n{\n\tpreg_r...,0,C,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,CWE-119,CVE-2015-8382,static PHP_FUNCTION(preg_replace_callback)\n{\...,0,C,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,CWE-119,CVE-2015-8382,static PHP_FUNCTION(preg_filter)\n{\n\tpreg_re...,0,C,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23827,CWE-119,CVE-2016-0835,IMPEG2D_ERROR_CODES_T impeg2d_dec_p_b_slice(de...,1,C,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,262
23828,CWE-119,CVE-2016-0835,void impeg2d_dec_p_mb_params(dec_state_t *ps_d...,1,C,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,170
23829,CWE-119,CVE-2016-0835,void impeg2d_dec_pnb_mb_params(dec_state_t *ps...,1,C,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,200
23830,CWE-125,CVE-2018-9503,"uint8_t rfc_parse_data(tRFC_MCB* p_mcb, MX_FRA...",1,C,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,107


In [7]:
df.columns

Index(['CWE ID', 'CVE ID', 'CODE', 'vul', 'lang', 'IDX', 'chmod()', 'assert()',
       'atoi()', 'atof()',
       ...
       'array_index_vulnerability', 'int', 'short', 'long', 'long long',
       'unsigned int', 'unsigned short', 'unsigned long', 'unsigned long long',
       'line_count'],
      dtype='object', length=106)

In [8]:
df = df.drop(columns="CWE ID")
df = df.drop(columns="CVE ID")
df = df.drop(columns="vul")

In [9]:
df.columns

Index(['CODE', 'lang', 'IDX', 'chmod()', 'assert()', 'atoi()', 'atof()',
       'calloc()', 'errno()', 'exit()',
       ...
       'array_index_vulnerability', 'int', 'short', 'long', 'long long',
       'unsigned int', 'unsigned short', 'unsigned long', 'unsigned long long',
       'line_count'],
      dtype='object', length=103)

In [10]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer



In [11]:
model = RobertaForSequenceClassification.from_pretrained("microsoft/unixcoder-base-nine", num_labels = 10)
tokenizer = RobertaTokenizer.from_pretrained("microsoft/unixcoder-base-nine")
batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/unixcoder-base-nine and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/938k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/444k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [12]:
def preprocess(df,file_name):
  #preprocess_df = df.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv(file_name)

def tokenized(examples):
  return tokenizer(examples['CODE'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [13]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='IDX',new_column_name='labels')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/23832 [00:00<?, ? examples/s]

In [14]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=100)
import torch
from torch.utils.data import DataLoader, Dataset, random_split
dataloader = DataLoader(encoded_dataset["test"], batch_size=1, shuffle=True)
random_samples = [data for i, data in enumerate(dataloader) if i < 100]


In [15]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np  # numpy를 임포트합니다.

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 정밀도를 계산합니다.
    precision_per_class = precision_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

    # 각 클래스에 대한 재현율, F1 스코어, 전체 정확도를 계산합니다.
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장합니다.
    metrics = {
        'accuracy': accuracy,
        'recall': recall,
        'f1': f1,
    }

    # 클래스별 정밀도를 딕셔너리에 추가합니다.
    for i, precision in enumerate(precision_per_class):
        metrics[f'precision_class_{i}'] = precision

    return metrics

In [24]:
import torch.nn as nn
class CustomModel(nn.Module):
    def __init__(self, model):
        super(CustomModel, self).__init__()
        self.model = model
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return loss, logits

In [25]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=7,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
)

trainer = Trainer(
    model=CustomModel(model),
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [26]:
gc.collect()
torch.cuda.empty_cache()

In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall,F1,Precision Class 0,Precision Class 1,Precision Class 2,Precision Class 3,Precision Class 4,Precision Class 5,Precision Class 6,Precision Class 7,Precision Class 8,Precision Class 9
1,0.102100,0.470163,0.920140,0.920140,0.919868,0.971247,0.881639,0.777228,0.771144,0.928230,0.864407,0.912913,0.921875,0.756522,0.678832
2,0.089600,0.451820,0.931608,0.931608,0.931280,0.975827,0.895057,0.951515,0.893082,0.905248,0.883621,0.820972,0.849398,0.844660,0.911765
3,0.066200,0.441020,0.930210,0.930210,0.930219,0.976667,0.930612,0.898876,0.923077,0.845850,0.890558,0.891738,0.712195,0.830000,0.836364
4,0.045900,0.448298,0.935944,0.935944,0.935672,0.976190,0.922276,0.904494,0.832461,0.872801,0.928571,0.869333,0.898089,0.814815,0.851852
5,0.031500,0.424546,0.936224,0.936224,0.936254,0.984239,0.900612,0.935673,0.913793,0.878116,0.887234,0.874667,0.900662,0.821429,0.815789
6,0.021500,0.452756,0.936084,0.936084,0.936157,0.982870,0.910296,0.909605,0.899441,0.898161,0.900217,0.842784,0.854545,0.818182,0.800000
7,0.012200,0.469437,0.937902,0.937902,0.938069,0.984520,0.924782,0.909605,0.889503,0.887500,0.888655,0.856021,0.859756,0.800000,0.806723


TrainOutput(global_step=7301, training_loss=0.050826093265902844, metrics={'train_runtime': 3089.9036, 'train_samples_per_second': 37.792, 'train_steps_per_second': 2.363, 'total_flos': 0.0, 'train_loss': 0.050826093265902844, 'epoch': 7.0})

In [ ]:
trainer.evaluate()